In [34]:
import os
import xmltodict
import Bio.UniProt.GOA as GOA
from ftplib import FTP
import gzip
import numpy as np

In [35]:
cur_path = os.getcwd()
print(cur_path)
data_folder = '../data'

/Users/xhy/Desktop/CL/ACS/Modules/R214/Practical/Main-Practical/hx255/code


In [36]:
gaf_uri = '/pub/databases/GO/goa/HUMAN/goa_human.gaf.gz'
gaf_name = gaf_uri.split('/')[-1]

# Check if the file exists already
gaf_path = os.path.join(data_folder, gaf_name)
if(not os.path.isfile(gaf_path)):
    # Login to FTP server
    ebi_ftp = FTP('ftp.ebi.ac.uk')
    ebi_ftp.login() # Logs in anonymously
    
    # Download
    print('Downloading GAF from GOA ftp site...')
    with open(gaf_path,'wb') as f:
        ebi_ftp.retrbinary('RETR {}'.format(gaf_uri), f.write)
        
    # Logout from FTP server
    ebi_ftp.quit()

In [37]:
# File is a gunzip file, so we need to open it in this way
with gzip.open(gaf_path, 'rt') as f:
    goa = {}  # Initialise the dictionary of functions
    
    # Iterate on each function using Bio.UniProt.GOA library.
    for entry in GOA.gafiterator(f):
        uniprot_id = entry.pop('DB_Object_ID')
        goa[uniprot_id] = entry

In [38]:
# construct dict mapping from env conditions to GO_ID
cond2go = {
    # ionizing radiation
    'cellular response to gamma radiation': 'GO:0071480',
    'response to x-ray': 'GO:0010165',
    # light stimulus
    'response to UV': 'GO:0009411', # further to UV_A/B/C
    # toxic substance
    'response to nicotine': 'GO:0035094',
    #'cellular response to benomyl': 'GO:0072755',
    #'response to amitrole': 'GO:0072722',
    #'response to paraquat': 'GO:1901562',
    # toxic
    'response to lead ion': 'GO:0010288',
    'response to cadmium ion': 'GO:0046686',
    'response to mercury ion': 'GO:0046689',
    'response to arsenic-containing substance': 'GO:0046685',
}
cond_list = ['cellular response to gamma radiation', 'response to x-ray', 'response to UV',
            'response to nicotine', 'response to lead ion', 'response to cadmium ion', 'response to mercury ion', 'response to arsenic-containing substance']

In [39]:
def invertdict(d):
    new_d = {v: k for k, v in d.items()}
    return new_d

go2cond = invertdict(cond2go)
print(go2cond)

{'GO:0071480': 'cellular response to gamma radiation', 'GO:0010165': 'response to x-ray', 'GO:0009411': 'response to UV', 'GO:0035094': 'response to nicotine', 'GO:0010288': 'response to lead ion', 'GO:0046686': 'response to cadmium ion', 'GO:0046689': 'response to mercury ion', 'GO:0046685': 'response to arsenic-containing substance'}


In [40]:
# preprocess cond_amigo_all.txt to cond_amigo_gene.txt
ori_file = os.path.join(data_folder, 'cond_amigo_all.txt')
amigo_file = os.path.join(data_folder, 'cond_amigo_gene.txt')
if (not os.path.isfile(amigo_file)):
    with open(ori_file, 'r') as fi:
        with open(amigo_file, 'w') as fo:
            for line in fi.readlines():
                split = line.strip().split('\t')
                print(split)
                if (split[4] in cond_list) and (split[11]=='gene'):
                    fo.write(line)

In [41]:
# construct dict recording association between condition GO_IDs and genes
cond_genes = {}
with open(amigo_file, 'r') as f:
    for line in f.readlines():
        split = line.strip().split('\t')
        if split[4] in go2cond.keys():
            if go2cond[split[4]] in cond_genes.keys():
                cond_genes[go2cond[split[4]]].add(split[2].upper())
            else:
                cond_genes[go2cond[split[4]]] = {split[2].upper()} # create a set


In [42]:
old_dis_list = ['lung neoplasms', 'diabetes mellitus', 'asthma', 'parkinson disease', 'obesity']
dis_list = ['lung neoplasms', 'skin neoplasms', 'diarrhea', 'diabetes mellitus', 'asthma', 'parkinson disease']
# disease appears in 'PheGenI_Association_full.tab'
# ori_phegeni_file = os.path.join(data_folder, 'PheGenI_Association_full.tab')

# disease appears in 'all_gene_disease_associations.tsv' from DisGeNET (more association entries)
ori_disgenet_file = os.path.join(data_folder, 'all_gene_disease_associations.tsv')

# preprocess phegeni file, extract useful lines
# phegeni_file = os.path.join(data_folder, 'PheGenI_Association_extracted.tab')
# with open(ori_phegeni_file, 'r') as fi:
#     with open(phegeni_file, 'w') as fo:
#         for line in fi.readlines():
#             split = line.strip().split('\t')
#             for dis in dis_list:
#                 if dis in split[1].lower(): # split[1] is 'Trait'
#                     split[1] = dis
#                     new_line = '\t'.join(split)
#                     fo.write(new_line)
#                     fo.write('\n')

# preprocess disgenet file, extract useful lines
disgenet_file = os.path.join(data_folder, 'extracted_gene_disease_associations.tsv')
with open(ori_disgenet_file, 'r', errors='replace') as fi:
    with open(disgenet_file, 'w') as fo:
        for line in fi.readlines():
            split = line.strip().split('\t')
            for dis in dis_list:
                if dis in split[3].lower(): # split[3] is 'trait'
                    split[3] = dis
                    new_line = '\t'.join(split)
                    print(new_line)
                    fo.write(new_line)
                    fo.write('\n')
        

10	NAT2	C0004096	asthma	0.0275762208930204	12	2	BEFREE;GAD
10	NAT2	C0011849	diabetes mellitus	0.00809642307981595	4	0	BEFREE;GAD;LHGDN
10	NAT2	C0011854	diabetes mellitus	0.0026817553075011	1	0	BEFREE;GAD
10	NAT2	C0011860	diabetes mellitus	0.0026817553075011	2	0	BEFREE;GAD
10	NAT2	C0024121	lung neoplasms	0.0178268514875933	7	0	GAD;LHGDN
10	NAT2	C0030567	parkinson disease	0.0320086512204254	16	0	BEFREE;GAD;LHGDN
10	NAT2	C0155877	asthma	0.0005494535684262	2	0	BEFREE
10	NAT2	C0264423	asthma	0.0002747267842131	1	0	BEFREE
10	NAT2	C4275179	parkinson disease	0.0005494535684262	2	0	BEFREE
100	ADA	C0004096	asthma	0.21105290517153	6	0	BEFREE;GAD;HPO;LHGDN
100	ADA	C0011849	diabetes mellitus	0.0016483607052786	6	0	BEFREE
100	ADA	C0011854	diabetes mellitus	0.0080452659225033	6	0	BEFREE;GAD
100	ADA	C0011860	diabetes mellitus	0.0067371445360677	8	0	BEFREE;GAD
100	ADA	C0011991	diarrhea	0.2	0	0	HPO
100	ADA	C0024121	lung neoplasms	0.2	1	0	CTD_human
100	ADA	C0342276	diabetes mellitus	0.0016483607052786	6	

102	ADAM10	C0011854	diabetes mellitus	0.0002747267842131	1	0	BEFREE
102	ADAM10	C0030567	parkinson disease	0.0002747267842131	1	0	BEFREE
1020	CDK5	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
1020	CDK5	C0011860	diabetes mellitus	0.0043301160127797	8	0	BEFREE;GAD
1020	CDK5	C0030567	parkinson disease	0.0013736339210655	5	0	BEFREE
10200	MPHOSPH6	C0030567	parkinson disease	0.002747267842131	10	0	BEFREE
1021	CDK6	C0011854	diabetes mellitus	0.002407028523288	1	0	GAD
1021	CDK6	C0030567	parkinson disease	0.0002747267842131	1	0	BEFREE
10212	DDX39A	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
10212	DDX39A	C0011854	diabetes mellitus	0.0002747267842131	1	0	BEFREE
10215	OLIG2	C0011854	diabetes mellitus	0.0002747267842131	1	0	BEFREE
10215	OLIG2	C0024121	lung neoplasms	0.0002747267842131	1	0	BEFREE
10217	CTDSPL	C0024121	lung neoplasms	0.00273291246481375	1	0	LHGDN
1022	CDK7	C0011860	diabetes mellitus	0.002407028523288	1	0	GAD
1022	CDK7	C0024121	lung neoplasms	0.0027329124648137

1440	CSF3	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
1440	CSF3	C0011991	diarrhea	0.2	1	0	CTD_human
1440	CSF3	C0024121	lung neoplasms	0.00273291246481375	1	0	LHGDN
1440	CSF3	C0037286	skin neoplasms	0.2	1	0	CTD_human
1441	CSF3R	C0004096	asthma	0.002407028523288	1	0	GAD
144125	OR2AG1	C0011860	diabetes mellitus	0.0002747267842131	1	0	BEFREE
144125	OR2AG1	C0011991	diarrhea	0.0005494535684262	2	0	BEFREE
144165	PRICKLE1	C0011860	diabetes mellitus	0.002407028523288	1	0	GAD
144233	BCDIN3D	C0004096	asthma	0.002407028523288	1	0	GAD
144233	BCDIN3D	C0011860	diabetes mellitus	0.002407028523288	1	0	GAD
1444	CSHL1	C0011860	diabetes mellitus	0.0002747267842131	1	0	BEFREE
1445	CSK	C0011854	diabetes mellitus	0.0002747267842131	1	0	BEFREE
1447	CSN2	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
1447	CSN2	C0011854	diabetes mellitus	0.0002747267842131	1	0	BEFREE
145258	GSC	C0011854	diabetes mellitus	0.002407028523288	1	0	GAD
145264	SERPINA12	C0011860	diabetes mellitus	0.0032312088759

2305	FOXM1	C0024121	lung neoplasms	0.201373633921065	6	0	BEFREE;CTD_human
2305	FOXM1	C0158981	diabetes mellitus	0.0002747267842131	1	0	BEFREE
23054	NCOA6	C0011849	diabetes mellitus	0.002407028523288	1	0	GAD
23054	NCOA6	C0011860	diabetes mellitus	0.08	1	0	RGD
23059	CLUAP1	C0024121	lung neoplasms	0.00273291246481375	1	0	LHGDN
2308	FOXO1	C0011849	diabetes mellitus	0.0037806624443535	6	0	BEFREE;GAD
2308	FOXO1	C0011860	diabetes mellitus	0.012375381935283	11	5	BEFREE;GAD
2308	FOXO1	C0030567	parkinson disease	0.0002747267842131	1	0	BEFREE
23082	PPRC1	C0011860	diabetes mellitus	0.002407028523288	1	0	GAD
2309	FOXO3	C0011853	diabetes mellitus	0.28	4	0	CTD_human;RGD
2309	FOXO3	C0024121	lung neoplasms	0.0002747267842131	1	0	BEFREE
2309	FOXO3	C0030567	parkinson disease	0.0002747267842131	1	0	BEFREE
231	AKR1B1	C0004096	asthma	0.0008241803526393	3	0	BEFREE
231	AKR1B1	C0011849	diabetes mellitus	0.00657908744379715	14	0	BEFREE;LHGDN
231	AKR1B1	C0011853	diabetes mellitus	0.08	1	0	RGD
231	AKR1B1	C0011854

2994	GYPB	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
29943	PADI1	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
29943	PADI1	C0011860	diabetes mellitus	0.0002747267842131	1	0	BEFREE
29949	IL19	C0004096	asthma	0.00678830169338035	7	0	BEFREE;GAD;LHGDN
29949	IL19	C0011854	diabetes mellitus	0.202407028523288	1	0	CTD_human;GAD
29956	CERS2	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
2996	GYPE	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
29965	CDIP1	C0004096	asthma	0.0002747267842131	1	0	BEFREE
29968	PSAT1	C0011860	diabetes mellitus	0.0002747267842131	1	0	BEFREE
2997	GYS1	C0011860	diabetes mellitus	0.0262680995065848	16	3	BEFREE;GAD
29979	UBQLN1	C0024121	lung neoplasms	0.0002747267842131	1	0	BEFREE
29979	UBQLN1	C0030567	parkinson disease	0.002407028523288	1	0	GAD
2998	GYS2	C0011860	diabetes mellitus	0.002407028523288	1	0	GAD
29985	SLC39A3	C0011860	diabetes mellitus	0.0002747267842131	1	0	BEFREE
29986	SLC39A2	C0004096	asthma	0.0002747267842131	1	

...

450093	LNCR1	C0024121	lung neoplasms	0.2	0	0	CTD_human
450095	PLF	C0004096	asthma	0.0115385249369502	42	0	BEFREE
450095	PLF	C0011849	diabetes mellitus	0.0008241803526393	3	0	BEFREE
450095	PLF	C0155877	asthma	0.0005494535684262	2	0	BEFREE
4502	MT2A	C0011849	diabetes mellitus	0.200549453568426	3	1	BEFREE;CTD_human
4502	MT2A	C0011860	diabetes mellitus	0.0032312088759273	4	1	BEFREE;GAD
4507	MTAP	C0011849	diabetes mellitus	0.002407028523288	1	0	GAD
4507	MTAP	C0011860	diabetes mellitus	0.002407028523288	1	0	GAD
4507	MTAP	C0037286	skin neoplasms	0.004814057046576	2	1	GAD
4508	ATP6	C0011849	diabetes mellitus	0.2	1	0	CTD_human
4508	ATP6	C0011860	diabetes mellitus	0.002407028523288	1	0	GAD
4508	ATP6	C0011991	diarrhea	0.0002747267842131	1	0	BEFREE
4508	ATP6	C0030567	parkinson disease	0.004814057046576	2	0	GAD
4509	ATP8	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
4509	ATP8	C0011860	diabetes mellitus	0.0026817553075011	2	0	BEFREE;GAD
4509	ATP8	C0030567	parkinson disease	0.0024070285232

5346	PLIN1	C0011860	diabetes mellitus	0.0008241803526393	3	1	BEFREE
5346	PLIN1	C1837792	diabetes mellitus	0.2	0	0	HPO
5347	PLK1	C0024121	lung neoplasms	0.0002747267842131	1	0	BEFREE
5350	PLN	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
5350	PLN	C0011860	diabetes mellitus	0.08	1	0	RGD
5350	PLN	C0342257	diabetes mellitus	0.08	1	0	RGD
5360	PLTP	C0011849	diabetes mellitus	0.0010989071368524	4	0	BEFREE
5360	PLTP	C0011860	diabetes mellitus	0.0053635106150022	4	1	BEFREE;GAD
5362	PLXNA2	C0011860	diabetes mellitus	0.0002747267842131	1	0	BEFREE
5362	PLXNA2	C0030567	parkinson disease	0.002407028523288	1	0	GAD
53630	BCO1	C0011860	diabetes mellitus	0.002407028523288	1	0	GAD
53632	PRKAG3	C0011849	diabetes mellitus	0.002407028523288	1	0	GAD
53632	PRKAG3	C0011860	diabetes mellitus	0.002407028523288	1	0	GAD
53637	S1PR5	C0011853	diabetes mellitus	0.2	1	0	CTD_human
5364	PLXNB1	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
5367	PMCH	C0004096	asthma	0.00273291246481375	1	0	LHGDN
5368

60529	ALX4	C0024121	lung neoplasms	0.2	1	0	CTD_human
6053	RNR2	C0030567	parkinson disease	0.002407028523288	1	0	GAD
6059	ABCE1	C0024121	lung neoplasms	0.0002747267842131	1	0	BEFREE
6060	RNU1-4	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
60685	ZFAND3	C0011860	diabetes mellitus	0.2	1	1	CTD_human
607	BCL9	C0011860	diabetes mellitus	0.0002747267842131	1	0	BEFREE
6091	ROBO1	C0004096	asthma	0.002407028523288	1	0	GAD
6091	ROBO1	C0024121	lung neoplasms	0.202732912464814	1	0	CTD_human;LHGDN
6093	ROCK1	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
6093	ROCK1	C0011860	diabetes mellitus	0.0002747267842131	1	0	BEFREE
6093	ROCK1	C0024121	lung neoplasms	0.00273291246481375	1	0	LHGDN
6094	ROM1	C0155877	asthma	0.0002747267842131	1	0	BEFREE
6097	RORC	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
6097	RORC	C0011860	diabetes mellitus	0.0002747267842131	1	0	BEFREE
6098	ROS1	C0004096	asthma	0.0002747267842131	1	0	BEFREE
6098	ROS1	C0011849	diabetes mellitus	0.00082418035263

6934	TCF7L2	C0024121	lung neoplasms	0.0054658249296275	2	0	LHGDN
6934	TCF7L2	C1960272	diabetes mellitus	0.0016483607052786	6	1	BEFREE
6935	ZEB1	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
6941	TCF19	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
6941	TCF19	C0011854	diabetes mellitus	0.0005494535684262	2	0	BEFREE
6941	TCF19	C0011860	diabetes mellitus	0.0002747267842131	1	0	BEFREE
6942	TCF20	C0011860	diabetes mellitus	0.016849199663016	7	0	GAD
6948	TCN2	C0011991	diarrhea	0.2	0	0	HPO
6949	TCOF1	C0030567	parkinson disease	0.0002747267842131	1	0	BEFREE
695	BTK	C0011991	diarrhea	0.2	0	0	HPO
695	BTK	C0401151	diarrhea	0.2	0	0	HPO
6955	TRA	C0004096	asthma	0.004814057046576	2	0	GAD
6957	TRB	C0004096	asthma	0.004814057046576	2	0	GAD
6957	TRB	C0011854	diabetes mellitus	0.0002747267842131	1	0	BEFREE
6962	TRBV20OR9-2	C0004096	asthma	0.0008241803526393	3	0	BEFREE
6962	TRBV20OR9-2	C0011849	diabetes mellitus	0.002747267842131	10	0	BEFREE
6962	TRBV20OR9-2	C0011854	diabetes mellitu

83756	TAS1R3	C0011860	diabetes mellitus	0.0002747267842131	1	0	BEFREE
83795	KCNK16	C0011860	diabetes mellitus	0.200274726784213	2	1	BEFREE;CTD_human
838	CASP5	C0024121	lung neoplasms	0.00513994098810175	2	0	GAD;LHGDN
83852	SETDB2	C0004096	asthma	0.002407028523288	1	0	GAD
83854	ANGPTL6	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
83855	KLF16	C0011860	diabetes mellitus	0.004814057046576	2	0	GAD
83856	FSD1L	C0004096	asthma	0.0002747267842131	1	0	BEFREE
83856	FSD1L	C0030567	parkinson disease	0.0002747267842131	1	0	BEFREE
83873	GPR61	C0011860	diabetes mellitus	0.0002747267842131	1	0	BEFREE
83879	CDCA7	C0011854	diabetes mellitus	0.0002747267842131	1	0	BEFREE
83886	PRSS27	C0011849	diabetes mellitus	0.0002747267842131	1	0	BEFREE
839	CASP6	C0030567	parkinson disease	0.0002747267842131	1	0	BEFREE
83933	HDAC10	C0024121	lung neoplasms	0.00273291246481375	1	0	LHGDN
83943	IMMP2L	C0011860	diabetes mellitus	0.0002747267842131	1	0	BEFREE
8395	PIP5K1B	C0004096	asthma	0.0002747267842131	1	1	B

In [59]:
# construct dict recording associations between diseases and genes
dis_genes = {}

with open(disgenet_file, 'r', errors='replace') as f:
    for line in f.readlines():
        split = line.strip().split('\t')
        if split[3] in dis_genes.keys():
            dis_genes[split[3]].add(split[1].upper())
            # dis_genes[split[1]].add(split[6].upper())
        else:
            dis_genes[split[3]] = {split[1].upper()}
            # dis_genes[split[1]].add(split[6].upper())
print(dis_genes)

{'asthma': {'MYOM2', 'HLA-DPA1', 'OPN3', 'IFNA8', 'CAV1', 'IL17F', 'PPP2CA', 'PNOC', 'SYT1', 'CXCL16', 'PTGER1', 'TP63', 'PDE4D', 'TAC4', 'PRH2', 'FTO', 'RRBP1', 'RBP7', 'SLC22A5', 'CKLF', 'MYH11', 'SLC22A2', 'CYP1A1', 'EPHB2', 'PC', 'CSMD1', 'CSE', 'LAT', 'ZFP36', 'CCL23', 'RUNX3', 'CCL18', 'ABCA1', 'RELA', 'CHIA', 'CFL1', 'CXCL10', 'HLA-C', 'FBRS', 'GIMAP4', 'SH2B1', 'IL4', 'TRAF6', 'IL15', 'HSPA1A', 'STAT5B', 'CDSN', 'ADORA1', 'SGCG', 'CTNNA3', 'RPLP0', 'HLA-DQB1', 'SLC52A2', 'HIST1H4C', 'CTNNBL1', 'EPHX2', 'FHL2', 'MIR19A', 'SLC22A1', 'CYP1A2', 'COX1', 'MAPK9', 'GSTK1', 'CDIP1', 'CMA1', 'CTLA4', 'SERPINB3', 'CHRM1', 'NTF4', 'TRAC', 'DCLK3', 'SERPINE2', 'UCN', 'KAT2B', 'PAX5', 'HIST1H4H', 'VCAN', 'S1PR1', 'MUSK', 'HLA-DQA2', 'IL13RA2', 'SLC52A1', 'CD28', 'GGTLC5P', 'KITLG', 'MAS1', 'ORMDL3', 'NM', 'CXCL12', 'NTF3', 'KIF3A', 'HIST1H4J', 'HIST1H4B', 'SULT1A1', 'ARSA', 'ABCD1', 'PRG2', 'LIPG', 'CCR6', 'RGS7BP', 'ACVR1', 'NOS1', 'IFNL2', 'SDF4', 'CCL24', 'PLB1', 'LINC01193', 'HIST1H4E',

In [44]:
# Jaccard index
def jaccard_index(set1, set2):
    n = len(set1.intersection(set2))
    return n / (len(set1) + len(set2) - n)

In [45]:
# direct (unenriched) Jaccard index between conditions and diseases
direct_jac = np.zeros((len(cond_list), len(dis_list)))
for cond_idx, cond in enumerate(cond_list):
    for dis_idx, dis in enumerate(dis_list):
        direct_jac[cond_idx][dis_idx] = jaccard_index(cond_genes[cond], dis_genes[dis])

for i in range(len(direct_jac)):
    print(direct_jac[i])
print()
#print(cond_genes['response to nicotine'])
#print(dis_genes['lung neoplasms'])
print(cond_list)
print(dis_list)
for i in range(len(direct_jac)):
    for j in range(len(direct_jac[i])):
        direct_jac[i][j] = direct_jac[i][j] * 50
for i in range(len(direct_jac)):
    print(direct_jac[i])

[ 0.00999167  0.0128866   0.00222717  0.00487465  0.00487126  0.00618375]
[ 0.01495017  0.01772152  0.01101322  0.00590688  0.00693001  0.00790167]
[ 0.01456311  0.03333333  0.00614754  0.00446122  0.00677966  0.00768574]
[ 0.02052545  0.02919708  0.00840336  0.01214856  0.01305842  0.0254386 ]
[ 0.01244813  0.01010101  0.01555556  0.00801394  0.00833912  0.01415929]
[ 0.01139138  0.01438849  0.01265823  0.00690608  0.00889802  0.00862813]
[ 0.00405515  0.00239808  0.0021097   0.00310131  0.00273038  0.00344828]
[ 0.00748752  0.00773196  0.01128668  0.00347947  0.00487805  0.00441696]

['cellular response to gamma radiation', 'response to x-ray', 'response to UV', 'response to nicotine', 'response to lead ion', 'response to cadmium ion', 'response to mercury ion', 'response to arsenic-containing substance']
['lung neoplasms', 'skin neoplasms', 'diarrhea', 'diabetes mellitus', 'asthma', 'parkinson disease']
[ 0.49958368  0.6443299   0.11135857  0.24373259  0.24356298  0.30918728]
[ 0.74

In [46]:
# download 'go-basic.obo'
from goatools import obo_parser
import wget
go_obo_url = 'http://purl.obolibrary.org/obo/go/go-basic.obo'
if(not os.path.isfile(data_folder+'/go-basic.obo')):
    go_obo = wget.download(go_obo_url, data_folder+'/go-basic.obo')
else:
    go_obo = data_folder+'/go-basic.obo'
go = obo_parser.GODag(go_obo)


load obo file ../data/go-basic.obo
../data/go-basic.obo: fmt(1.2) rel(2018-04-14) 47,199 GO Terms


In [47]:
def common_parent_go_ids(terms, go):
    '''
        This function finds the common ancestors in the GO 
        tree of the list of terms in the input.
    '''
    # Find candidates from first
    rec = go[terms[0]]
    candidates = rec.get_all_parents()
    candidates.update({terms[0]})
    
    # Find intersection with second to nth term
    for term in terms[1:]:
        rec = go[term]
        parents = rec.get_all_parents()
        parents.update({term})
        
        # Find the intersection with the candidates, and update.
        candidates.intersection_update(parents)
        
    return candidates

def deepest_common_ancestor(terms, go):
    return max(common_parent_go_ids(terms, go), key=lambda t: go[t].depth)

# finds the minimum branch length between two terms in the GO DAG
def min_branch_length(go_id1, go_id2, go):
    '''
        Finds the minimum branch length between two terms in the GO DAG.
    '''
    # First get the deepest common ancestor
    dca = deepest_common_ancestor([go_id1, go_id2], go)
    
    # Then get the distance from the DCA to each term
    dca_depth = go[dca].depth
    d1 = go[go_id1].depth - dca_depth
    d2 = go[go_id2].depth - dca_depth
    
    # Return the total distance - i.e., to the deepest common ancestor and back.
    return d1 + d2

In [48]:
# semantic similarity (1st option) between conditions
def semantic_similarity(go_id1, go_id2, go):
    dist = min_branch_length(go_id1, go_id2, go)
    if dist == 0:
        return -1.0
    return 1.0 / dist

In [49]:
# calculate semantic similarity
n = len(cond_list)
cond_res_sim = np.zeros((n, n))
for i in range(n):
    go_id1 = cond2go[cond_list[i]]
    for j in range(n):        
        go_id2 = cond2go[cond_list[j]]
        cond_res_sim[i][j] = semantic_similarity(go_id1, go_id2, go)
        print(cond_list[i], cond_list[j])
        print(cond_res_sim[i][j])
print(cond_res_sim)   

cellular response to gamma radiation cellular response to gamma radiation
-1.0
cellular response to gamma radiation response to x-ray
0.25
cellular response to gamma radiation response to UV
0.166666666667
cellular response to gamma radiation response to nicotine
0.111111111111
cellular response to gamma radiation response to lead ion
0.1
cellular response to gamma radiation response to cadmium ion
0.1
cellular response to gamma radiation response to mercury ion
0.1
cellular response to gamma radiation response to arsenic-containing substance
0.125
response to x-ray cellular response to gamma radiation
0.25
response to x-ray response to x-ray
-1.0
response to x-ray response to UV
0.25
response to x-ray response to nicotine
0.142857142857
response to x-ray response to lead ion
0.125
response to x-ray response to cadmium ion
0.125
response to x-ray response to mercury ion
0.125
response to x-ray response to arsenic-containing substance
0.166666666667
response to UV cellular response to g

In [50]:
import math
from collections import Counter

class TermCounts():
    '''
        TermCounts counts the term counts for each 
    '''
    def __init__(self, go, annots):
        '''
            Initialise the counts and  
        '''
        # Backup
        self._go = go
        
        # Initialise the counters
        self._counts = Counter()
        self._aspect_counts = Counter()
        
        # Fill the counters...
        self._count_terms(go, annots)
        
    def _count_terms(self, go, annots):
        '''
            Fills in the counts and overall aspect counts.
        '''
        for x in annots:
            # Extract term information
            go_id = annots[x]['GO_ID']
            namespace = go[go_id].namespace

            self._counts[go_id] += 1
            rec = go[go_id]
            parents = rec.get_all_parents()
            for p in parents:
                self._counts[p] += 1
            
            self._aspect_counts[namespace] += 1
            
    def get_count(self, go_id):
        '''
            Returns the count of that GO term observed in the annotations.
        '''
        return self._counts[go_id]
        
    def get_total_count(self, aspect):
        '''
            Gets the total count that's been precomputed.
        '''
        return self._aspect_counts[aspect]
    
    def get_term_freq(self, go_id):
        '''
            Returns the frequency at which a particular GO term has 
            been observed in the annotations.
        '''
        try:
            namespace = self._go[go_id].namespace
            freq = float(self.get_count(go_id)) / float(self.get_total_count(namespace))
        except ZeroDivisionError:
            freq = 0
        
        return freq

termcounts = TermCounts(go, goa)

In [51]:
# information content (IC)
def ic(go_id, termcounts):
    # Get the observed frequency of the GO term
    freq = termcounts.get_term_freq(go_id)
    if freq == 0:
        return -1.0 # not appear
    # Calculate the information content (i.e., -log("freq of GO term")
    return -1.0 * math.log(freq)

In [52]:
# Resnik's similarity measure: the information content of the most informative common ancestor, i.e. the most specific common parent-term in the GO 
def resnik_sim(go_id1, go_id2, go, termcounts):
    msca = deepest_common_ancestor([go_id1, go_id2], go)
    return ic(msca, termcounts)

In [53]:
# test
go_id3 = 'GO:0071480'
go_id4 = 'GO:0009411'
go_id5 = 'GO:0010288'
sim_r = resnik_sim(go_id3, go_id4, go, termcounts)
print('Resnik similarity score 1: {}'.format(sim_r))
sim_r = resnik_sim(go_id3, go_id5, go, termcounts)
print('Resnik similarity score 1: {}'.format(sim_r))

Resnik similarity score 1: 6.020052003959894
Resnik similarity score 1: 2.1249717192620605


In [54]:
n = len(cond_list)
cond_res_sim = np.zeros((n, n))
for i in range(n):
    go_id1 = cond2go[cond_list[i]]
    for j in range(n):        
        go_id2 = cond2go[cond_list[j]]
        cond_res_sim[i][j] = resnik_sim(go_id1, go_id2, go, termcounts)
        print(cond_list[i], cond_list[j])
        print(cond_res_sim[i][j])
print(cond_res_sim)    

cellular response to gamma radiation cellular response to gamma radiation
7.62948991639
cellular response to gamma radiation response to x-ray
6.84103255603
cellular response to gamma radiation response to UV
6.02005200396
cellular response to gamma radiation response to nicotine
2.12497171926
cellular response to gamma radiation response to lead ion
2.12497171926
cellular response to gamma radiation response to cadmium ion
2.12497171926
cellular response to gamma radiation response to mercury ion
2.12497171926
cellular response to gamma radiation response to arsenic-containing substance
2.12497171926
response to x-ray cellular response to gamma radiation
6.84103255603
response to x-ray response to x-ray
-1.0
response to x-ray response to UV
6.02005200396
response to x-ray response to nicotine
2.12497171926
response to x-ray response to lead ion
2.12497171926
response to x-ray response to cadmium ion
2.12497171926
response to x-ray response to mercury ion
2.12497171926
response to x-ra

In [55]:
# compartment
cond_comps = {}
for cond in cond_list:
    cond_comps[cond] = list()

comppi_file = os.path.join(data_folder, 'comppi_proteins_locs.txt')
with open(comppi_file, 'r') as f:
    for line in f.readlines():
        split = line.strip().split('\t')
        for cond, genes in cond_genes.items():
            for gene in genes:
                if gene in split[2]:
                    loc = split[3].split(':')[0]
                    cond_comps[cond].append(loc)
                    break
print(cond_comps)
            

{'cellular response to gamma radiation': ['cytosol', 'cytosol', 'cytosol', 'cytosol', 'membrane', 'mitochondrion', 'cytosol', 'nucleus', 'nucleus', 'cytosol', 'cytosol', 'cytosol', 'nucleus', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'mitochondrion', 'cytosol', 'nucleus', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'nucleus', 'cytosol', 'cytosol', 'cytosol', 'secretory-pathway', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'nucleus', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'nucleus', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'nucleus', 'mitochondrion', 'cytosol', 'nucleus', 'cytosol', 'nucleus', 'nucleus', 'cytosol', 'cytosol', 'nucleus', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'cytosol', 'nucleus', 'nucleus', 'cytosol', 'mito

In [56]:
# calculate how many compartments appeared
comp_set = set()
for cond, comps in cond_comps.items():
    for comp in comps:
        comp_set.add(comp)
print(comp_set)

{'secretory-pathway', 'nucleus', 'mitochondrion', 'membrane', 'cytosol', 'extracellular'}


In [57]:
cond_comp_counts = {}
comp_list = ['nucleus', 'membrane', 'cytosol', 'mitochondrion', 'extracellular', 'secretory-pathway']
for cond, comps in cond_comps.items():
    counts = []
    for comp in comp_list:
        count = comps.count(comp)
        counts.append(count)
    cond_comp_counts[cond] = counts
print(cond_comp_counts)

{'cellular response to gamma radiation': [67, 31, 201, 16, 2, 1], 'response to x-ray': [131, 99, 376, 31, 10, 13], 'response to UV': [109, 117, 326, 19, 20, 4], 'response to nicotine': [173, 364, 585, 73, 29, 29], 'response to lead ion': [374, 218, 759, 79, 17, 20], 'response to cadmium ion': [54, 450, 407, 59, 6, 13], 'response to mercury ion': [476, 563, 1216, 111, 22, 58], 'response to arsenic-containing substance': [10, 19, 70, 7, 0, 1]}


In [58]:
cond_comp_ratios = {}
new_comp_list = ['nucleus', 'membrane', 'cytosol', 'mitochondrion', 'extracellular']

for cond in cond_comps:
    ratios = []
    counts = cond_comp_counts[cond]
    total_count = 0
    for i in range(len(counts)-1):
        total_count += counts[i]
    for i in range(len(counts)-1):
        ratios.append(counts[i]/total_count)
    cond_comp_ratios[cond] = ratios
for cond, ratios in cond_comp_ratios.items():
    print(cond)
    i = 0
    for ratio in ratios:
        print(new_comp_list[i], '{0:.3f}'.format(ratio))
        i += 1
    print()
print(cond_comp_ratios)

cellular response to gamma radiation
nucleus 0.211
membrane 0.098
cytosol 0.634
mitochondrion 0.050
extracellular 0.006

response to x-ray
nucleus 0.202
membrane 0.153
cytosol 0.581
mitochondrion 0.048
extracellular 0.015

response to UV
nucleus 0.184
membrane 0.198
cytosol 0.552
mitochondrion 0.032
extracellular 0.034

response to nicotine
nucleus 0.141
membrane 0.297
cytosol 0.478
mitochondrion 0.060
extracellular 0.024

response to lead ion
nucleus 0.258
membrane 0.151
cytosol 0.525
mitochondrion 0.055
extracellular 0.012

response to cadmium ion
nucleus 0.055
membrane 0.461
cytosol 0.417
mitochondrion 0.060
extracellular 0.006

response to mercury ion
nucleus 0.199
membrane 0.236
cytosol 0.509
mitochondrion 0.046
extracellular 0.009

response to arsenic-containing substance
nucleus 0.094
membrane 0.179
cytosol 0.660
mitochondrion 0.066
extracellular 0.000

{'cellular response to gamma radiation': [0.2113564668769716, 0.09779179810725552, 0.6340694006309149, 0.050473186119873815, 0.